In [1]:
import pathlib
import pickle

import numpy as np
import scipy.sparse
import scipy.io
import pandas as pd

In [2]:
save_prefix = 'E:/pathwayGNN/data/preprocessed/LNCD_processed/'

In [4]:
mirna_disease = pd.read_csv('E:/pathwayGNN/data/raw/LNCD/mirna_disease.dat', encoding='utf-8', delimiter='\t', names=['mirnaID', 'diseaseID', 'weight'])
lncrna_disease = pd.read_csv('E:/pathwayGNN/data/raw/LNCD/lncrna_disease.dat', encoding='utf-8', delimiter='\t', names=['lncrnaID', 'diseaseID', 'weight'])
lncrna_mirna = pd.read_csv('E:/pathwayGNN/data/raw/LNCD/lncrna_mirna.dat', encoding='utf-8', delimiter='\t', names=['lncrnaID', 'mirnaID', 'weight'])
num_mirna = 1842
num_lncrna = 6583
num_disease = 1190

In [5]:
train_val_test_idx = np.load('E:/pathwayGNN/data/raw/LNCD/example.npz')
train_idx = train_val_test_idx['train_idx']
val_idx = train_val_test_idx['val_idx']
test_idx = train_val_test_idx['test_idx']

lncrna_disease = lncrna_disease.loc[train_idx].reset_index(drop=True)



In [6]:
dim = num_mirna + num_lncrna + num_disease

type_mask = np.zeros((dim), dtype=int)
type_mask[num_lncrna:num_lncrna+num_disease] = 1
type_mask[num_lncrna+num_disease:] = 2

adjM = np.zeros((dim, dim), dtype=int)#创建一个维度为dim*dim,类型为Int，用0填充的数组
for _, row in lncrna_disease.iterrows():
    lid = row['lncrnaID'] - 1
    did = num_lncrna + row['diseaseID'] - 1
    adjM[lid, did] += 1
    adjM[did, lid] += 1
for _, row in lncrna_mirna.iterrows():
    lid = row['lncrnaID'] - 1
    mid = num_lncrna + num_disease + row['mirnaID'] - 1
    adjM[lid, mid] += 1
    adjM[mid, lid] += 1    
for _, row in mirna_disease.iterrows():
    mid = num_lncrna + num_disease + row['mirnaID'] - 1
    did = num_lncrna + row['diseaseID'] - 1
    adjM[mid, did] += 1
    adjM[did, mid] += 1 

In [7]:
lncrna_disease_list = {i: adjM[i, num_lncrna:num_lncrna+num_disease].nonzero()[0] for i in range(num_lncrna)}
disease_lncrna_list = {i: adjM[num_lncrna + i, :num_lncrna].nonzero()[0] for i in range(num_disease)}
disease_mirna_list = {i: adjM[num_lncrna + i, num_lncrna+num_disease:].nonzero()[0] for i in range(num_disease)}
mirna_disease_list = {i: adjM[num_lncrna + num_disease + i, num_lncrna:num_lncrna+num_disease].nonzero()[0] for i in range(num_mirna)}
lncrna_mirna_list = {i: adjM[i, num_lncrna+num_disease:].nonzero()[0] for i in range(num_lncrna)}
mirna_lncrna_list = {i: adjM[num_lncrna + num_disease + i, :num_lncrna].nonzero()[0] for i in range(num_mirna)}

In [8]:
# 0-1-0
l_d_l = []
for d, l_list in disease_lncrna_list.items():
    l_d_l.extend([(l1, d, l2) for l1 in l_list for l2 in l_list])
l_d_l = np.array(l_d_l)
l_d_l[:, 1] += num_lncrna
sorted_index = sorted(list(range(len(l_d_l))), key=lambda i : l_d_l[i, [0, 2, 1]].tolist())
l_d_l = l_d_l[sorted_index]

#0-2-0
l_m_l = []
for m, l_list in mirna_lncrna_list.items():
    l_m_l.extend([(l1, m, l2) for l1 in l_list for l2 in l_list])
l_m_l = np.array(l_m_l)

l_m_l[:, 1] += (num_lncrna + num_disease)
sorted_index = sorted(list(range(len(l_m_l))), key=lambda i : l_m_l[i, [0, 2, 1]].tolist())
l_m_l = l_m_l[sorted_index]
#2-1-2
m_d_m = []
for d, m_list in disease_mirna_list.items():
    m_d_m.extend([(m1, d, m2) for m1 in m_list for m2 in m_list])
m_d_m = np.array(m_d_m)
m_d_m += num_lncrna
m_d_m[:, [0, 2]] += num_disease
sorted_index = sorted(list(range(len(m_d_m))), key=lambda i : m_d_m[i, [0, 2, 1]].tolist())
m_d_m = m_d_m[sorted_index]
#0-2-1-2-0
l_m_d_m_l = []
for m1, d, m2 in m_d_m:
    if len(mirna_lncrna_list[m1 - num_lncrna - num_disease]) == 0 or len(mirna_lncrna_list[m2 - num_lncrna - num_disease]) == 0:
        continue
    candidate_l1_list = np.random.choice(len(mirna_lncrna_list[m1 - num_lncrna - num_disease]), int(0.2 * len(mirna_lncrna_list[m1 - num_lncrna - num_disease])), replace=False)
    candidate_l1_list = mirna_lncrna_list[m1 - num_lncrna - num_disease][candidate_l1_list]
    candidate_l2_list = np.random.choice(len(mirna_lncrna_list[m2 - num_lncrna - num_disease]), int(0.2 * len(mirna_lncrna_list[m2 - num_lncrna - num_disease])), replace=False)
    candidate_l2_list = mirna_lncrna_list[m2 - num_lncrna - num_disease][candidate_l2_list]    
    l_m_d_m_l.extend([(l1, m1, d, m2, l2) for l1 in candidate_l1_list for l2 in candidate_l2_list])
l_m_d_m_l = np.array(l_m_d_m_l)
sorted_index = sorted(list(range(len(l_m_d_m_l))), key=lambda i : l_m_d_m_l[i, [0, 4, 1, 2, 3]].tolist())
l_m_d_m_l = l_m_d_m_l[sorted_index]

#1_0_1
d_l_d = []
for l, d_list in lncrna_disease_list.items():
    d_l_d.extend([(d1, l, d2) for d1 in d_list for d2 in d_list])
d_l_d = np.array(d_l_d)
d_l_d[:, [0, 2]] += num_lncrna
sorted_index = sorted(list(range(len(d_l_d))), key=lambda i : d_l_d[i, [0, 2, 1]].tolist())
d_l_d = d_l_d[sorted_index]

#1_2_1
d_m_d = []
for m, d_list in mirna_disease_list.items():
    d_m_d.extend([(d1, m, d2) for d1 in d_list for d2 in d_list])
d_m_d = np.array(d_m_d)
d_m_d[:, 1] += (num_lncrna + num_disease)
d_m_d[:, [0, 2]] += num_lncrna
sorted_index = sorted(list(range(len(d_m_d))), key=lambda i : d_m_d[i, [0, 2, 1]].tolist())
d_m_d = d_m_d[sorted_index]


In [9]:
expected_metapaths = [
    [(0, 1, 0), (0, 2, 1, 2, 0), (0, 2 , 0)],
    [(1, 0, 1), (1, 2, 1)]
]
# create the directories if they do not exist
for i in range(len(expected_metapaths)):
    pathlib.Path(save_prefix + '{}'.format(i)).mkdir(parents=True, exist_ok=True)

metapath_indices_mapping = {(0, 1, 0): l_d_l,
                            (0, 2, 1, 2, 0): l_m_d_m_l,
                            (0, 2, 0): l_m_l,
                            (1, 0, 1): d_l_d,
                            (1, 2, 1): d_m_d}

# write all things 
target_idx_lists = [np.arange(num_lncrna), np.arange(num_disease)]
offset_list = [0, num_lncrna]
for i, metapaths in enumerate(expected_metapaths):
    for metapath in metapaths:
        edge_metapath_idx_array = metapath_indices_mapping[metapath]
        
        with open(save_prefix + '{}/'.format(i) + '-'.join(map(str, metapath)) + '_idx.pickle', 'wb') as out_file:
            target_metapaths_mapping = {}
            left = 0
            right = 0
            for target_idx in target_idx_lists[i]:
                while right < len(edge_metapath_idx_array) and edge_metapath_idx_array[right, 0] == target_idx + offset_list[i]:
                    right += 1
                target_metapaths_mapping[target_idx] = edge_metapath_idx_array[left:right, ::-1]
                left = right
            pickle.dump(target_metapaths_mapping, out_file)

        #np.save(save_prefix + '{}/'.format(i) + '-'.join(map(str, metapath)) + '_idx.npy', edge_metapath_idx_array)
        
        with open(save_prefix + '{}/'.format(i) + '-'.join(map(str, metapath)) + '.adjlist', 'w') as out_file:
            left = 0
            right = 0
            for target_idx in target_idx_lists[i]:
                while right < len(edge_metapath_idx_array) and edge_metapath_idx_array[right, 0] == target_idx + offset_list[i]:
                    right += 1
                neighbors = edge_metapath_idx_array[left:right, -1] - offset_list[i]
                neighbors = list(map(str, neighbors))
                if len(neighbors) > 0:
                    out_file.write('{} '.format(target_idx) + ' '.join(neighbors) + '\n')
                else:
                    out_file.write('{}\n'.format(target_idx))
                left = right

scipy.sparse.save_npz(save_prefix + 'adjM.npz', scipy.sparse.csr_matrix(adjM))
np.save(save_prefix + 'node_types.npy', type_mask)

In [10]:
# output lncrna_disease.npy
lncrna_disease = pd.read_csv('E:/pathwayGNN/data/raw/LNCD/lncrna_disease.dat', encoding='utf-8', delimiter='\t', names=['lncrnaID', 'diseaseID', 'weight'])
lncrna_disease = lncrna_disease[['lncrnaID', 'diseaseID']].to_numpy()
lncrna_disease = lncrna_disease - 1
np.save(save_prefix + 'lncrna_disease.npy', lncrna_disease)

In [11]:
# output positive and negative samples for training, validation and testing

np.random.seed(453289)
save_prefix = 'E:/pathwayGNN/data/preprocessed/LNCD_processed/'
num_lncrna = 6583
num_disease = 1191
lncrna_disease = np.load('E:/pathwayGNN/data/preprocessed/LNCD_processed/lncrna_disease.npy')
train_val_test_idx = np.load('E:/pathwayGNN/data/raw/LNCD/example.npz')
train_idx = train_val_test_idx['train_idx']
val_idx = train_val_test_idx['val_idx']
test_idx = train_val_test_idx['test_idx']

neg_candidates = []
counter = 0
for i in range(num_lncrna):
    for j in range(num_disease):
        if counter < len(lncrna_disease):
            if i == lncrna_disease[counter, 0] and j == lncrna_disease[counter, 1]:
                counter += 1
            else:
                neg_candidates.append([i, j])
        else:
            neg_candidates.append([i, j])
neg_candidates = np.array(neg_candidates)

idx = np.random.choice(len(neg_candidates), len(val_idx) + len(test_idx), replace=False)
val_neg_candidates = neg_candidates[sorted(idx[:len(val_idx)])]
test_neg_candidates = neg_candidates[sorted(idx[len(val_idx):])]

train_lncrna_disease = lncrna_disease[train_idx]
train_neg_candidates = []
counter = 0
for i in range(num_lncrna):
    for j in range(num_disease):
        if counter < len(train_lncrna_disease):
            if i == train_lncrna_disease[counter, 0] and j == train_lncrna_disease[counter, 1]:
                counter += 1
            else:
                train_neg_candidates.append([i, j])
        else:
            train_neg_candidates.append([i, j])
train_neg_candidates = np.array(train_neg_candidates)

np.savez(save_prefix + 'train_val_test_neg_lncrna_disease.npz',
         train_neg_lncrna_disease=train_neg_candidates,
         val_neg_lncrna_disease=val_neg_candidates,
         test_neg_lncrna_disease=test_neg_candidates)
np.savez(save_prefix + 'train_val_test_pos_lncrna_disease.npz',
         train_pos_lncrna_disease=lncrna_disease[train_idx],
         val_pos_lncrna_disease=lncrna_disease[val_idx],
         test_pos_lncrna_disease=lncrna_disease[test_idx])